In [1]:
import pandas as pd
import pandas_ml as pdml
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA


In [9]:
#Load data
df = pd.read_csv('data.csv');
#Replace NaN with 0
df = df.fillna(0);
df['DATE'] = pd.to_datetime(df['DATE']);
df['Fog/Ice']

0       0.0
1       0.0
2       0.0
3       1.0
4       0.0
5       1.0
6       1.0
7       1.0
8       1.0
9       1.0
10      0.0
11      0.0
12      0.0
13      0.0
14      1.0
15      1.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
       ... 
2880    0.0
2881    0.0
2882    0.0
2883    1.0
2884    1.0
2885    1.0
2886    1.0
2887    0.0
2888    0.0
2889    0.0
2890    0.0
2891    1.0
2892    1.0
2893    1.0
2894    1.0
2895    1.0
2896    0.0
2897    0.0
2898    0.0
2899    0.0
2900    0.0
2901    0.0
2902    1.0
2903    1.0
2904    1.0
2905    1.0
2906    0.0
2907    0.0
2908    0.0
2909    0.0
Name: Fog/Ice, Length: 2910, dtype: float64

In [3]:
#Add columns for wind squared
df['Avg Wind Speed Squared'] = pd.Series(np.square(df['Average Wind Speed']), index=df.index)
df['5 second wind speed squared'] = pd.Series(np.square(df['Fastest 5 second wind speed']));
df['2 min wind speed squared'] = pd.Series(np.square(df['Fastest 2 minute wind speed']));
df = df[['Power Outage', 'PRCP', 'TMAX','TMIN','Fastest 5 second wind speed','5 second wind speed squared', 'Fastest 2 minute wind speed' ,'2 min wind speed squared','Average Wind Speed','Avg Wind Speed Squared','Fog/Ice', 'Heavy/Freezing Fog', 'Thunder','Smoke/Haze']];


In [4]:
#Under sample based on Cluster Centroids
df_ML = pdml.ModelFrame(df, target = 'Power Outage')
df_ML.target_name
print(df_ML.target.value_counts());
sampler = df_ML.imbalance.under_sampling.ClusterCentroids();
sampled = df_ML.fit_sample(sampler);
print(sampled.target.value_counts());
finalData = pd.DataFrame(sampled);
finalData.head()

0.0    2751
1.0     159
Name: Power Outage, dtype: int64
1.0    159
0.0    159
Name: Power Outage, dtype: int64


,Power Outage,PRCP,TMAX,TMIN,Fastest 5 second wind speed,5 second wind speed squared,Fastest 2 minute wind speed,2 min wind speed squared,Average Wind Speed,Avg Wind Speed Squared,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze
0,0.0,0.02725,49.4,27.925,23.675,560.6575,20.0375,401.63375,8.1815,70.66704,0.25,0.125,0.025,0.15
1,0.0,0.29000,41.0,34.000,140.000,19600.0000,48.1000,2313.61000,2.6800,7.18240,1.00,1.000,0.000,0.00
2,0.0,0.02500,76.5,53.500,83.000,6889.0000,36.0000,1296.81000,6.1500,39.33540,1.00,0.500,0.000,1.00
3,0.0,0.38000,41.0,34.000,161.100,25953.2100,83.0000,6889.00000,8.2800,68.55840,1.00,0.000,0.000,1.00
4,0.0,0.00000,39.0,21.000,181.000,32761.0000,25.1000,630.01000,3.1300,9.79690,1.00,0.000,0.000,1.00


In [5]:
# Random Split
#Split the data in two parts.
# powerOutages = df.loc[df['Power Outage'] == 1];
# noPowerOutages = df.loc[df['Power Outage'] == 0];

# zeroDays = noPowerOutages.sample(159);
# finalData = pd.concat([powerOutages,zeroDays]);
# finalData = finalData.sample(318);


In [6]:
#Normalize data
scaledData = preprocessing.MinMaxScaler().fit_transform(finalData.values);
finalData = pd.DataFrame(scaledData, index = finalData.index, columns = finalData.columns);

In [7]:
finalData.to_csv('Clustered Data', index = False);